# Téléchargement depuis le FTP INPI
L'objectif de ce notebook est de récupérer la donnée de l'INPI provenant du FTP.

Le téléchargement depuis le FTP INPI a plusieurs contraintes :

la connexion FTP est bloquée par le proxy sur un poste standard CALF.
il y a un quota de téléchargement de 3Go par jour.
la connexion est coupée automatiquement par l'hôte au bout d'une centaine de fichiers récupérés via un script.
l'arborescence dans le FTP est profonde et pas incrémentale car liée à la codification des greffes. Ex : /public /IMR_Donnees_Saisies
  /tc
      /flux
          /2017
              /08
                  /04
                      /0101
                          /57
                      /0202
                          /60
                      /0203
                          /58
                      /0301
                          /58
                      ..etc


Ce notebook est basé sur la librairie ftputil et un parcours de l'arborescence du FTP avec un walk. Attention, étant donné la coupure au bout d'une centaine de fichiers, on ne peut pas facilement télécharger l'intégralité des fichiers avec ce notebook.

Fonctions :
- Se connecter au FTP
- Télécharger un fichier depuis le FTP
- Télécharger tous les fichiers d'un dossier spécifique depuis le FTP
- Lister tous les fichiers du FTP / d'un dossier spécifique

# Connect to FTP

In [ ]:
# INPI FTP Connexion parameters
host = 'opendata-rncs.inpi.fr'
port = 21
usr = 'calf'
pwd = 'Calfinpi2019'
path_local = "/home/ec2-user/SageMaker/Extract_inpi_FTS/temp"

In [ ]:
!pip install ftputil

In [ ]:
import os
import ftplib
import ftputil
import ftputil.session
#https://ftputil.sschwarzer.net/trac/wiki/Documentation

# Enable secure session
my_session_factory = ftputil.session.session_factory(
                       base_class=ftplib.FTP_TLS,
                       port=port,
                       encrypt_data_channel=True)

# create FTP Host
ftp_host = ftputil.FTPHost(host, usr, pwd,session_factory=my_session_factory)

In [ ]:
path='/public/IMR_Donnees_Saisies/tc/stock/2017'

# List files in FTP Directory

In [ ]:
# List files in FTP Directory

list_files=[]
for root,dirs,files in ftp_host.walk(path):
    for file in files:
          if file.endswith(('.csv','.zip')):
                list_files.append((root, file))
list_files[:5]

In [ ]:
print (str(len(list_files)) + ' files in ' + path)

# Download single file

In [ ]:
# Download file
import os
dest_path = 'dl'
(root, file)= list_files[0] 
source = os.path.join(root, file)
target = os.path.join(dest_path, file)
print(source)
print(target)

ftp_host.download(source, target)

# Download all files in a given path

In [ ]:
# Download all files in a given path
import os
dest_path = 'dl'
for root, file in list_files:
    source = os.path.join(root, file)
    target = os.path.join(dest_path, file)
    ftp_host.download(source, target)